# Ajuste Fino de Modelos Open AI

Este notebook baseia-se na orientação atual fornecida na documentação de [Ajuste Fino](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) da Open AI.

O ajuste fino melhora o desempenho dos modelos base para a sua aplicação, re-treinando-o com dados adicionais e contexto relevante para esse caso de uso ou cenário específico. Note que técnicas de engenharia de prompt como _few shot learning_ e _retrieval augmented generation_ permitem-lhe melhorar o prompt padrão com dados relevantes para aumentar a qualidade. No entanto, estas abordagens são limitadas pelo tamanho máximo da janela de tokens do modelo base alvo.

Com o ajuste fino, estamos efetivamente a re-treinar o próprio modelo com os dados necessários (permitindo-nos usar muitos mais exemplos do que cabem na janela máxima de tokens) - e a implementar uma versão _personalizada_ do modelo que já não precisa de ter exemplos fornecidos no momento da inferência. Isto não só melhora a eficácia do nosso design de prompt (temos mais flexibilidade em usar a janela de tokens para outras coisas) como potencialmente também melhora os nossos custos (reduzindo o número de tokens que precisamos enviar ao modelo no momento da inferência).

O ajuste fino tem 4 passos:
1. Preparar os dados de treino e carregá-los.
1. Executar o trabalho de treino para obter um modelo ajustado.
1. Avaliar o modelo ajustado e iterar para qualidade.
1. Implementar o modelo ajustado para inferência quando estiver satisfeito.

Note que nem todos os modelos base suportam ajuste fino - [verifique a documentação da OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) para a informação mais recente. Também pode ajustar um modelo previamente ajustado. Neste tutorial, usaremos `gpt-35-turbo` como o nosso modelo base alvo para ajuste fino.

---


### Passo 1.1: Prepare o Seu Conjunto de Dados

Vamos construir um chatbot que o ajude a compreender a tabela periódica dos elementos respondendo a perguntas sobre um elemento com um limerique. Neste _simples_ tutorial, vamos apenas criar um conjunto de dados para treinar o modelo com alguns exemplos de respostas que mostram o formato esperado dos dados. Num caso de uso real, seria necessário criar um conjunto de dados com muitos mais exemplos. Também poderá ser possível usar um conjunto de dados aberto (para o seu domínio de aplicação) se existir, e reformata-lo para uso no fine-tuning.

Como estamos a focar-nos no `gpt-35-turbo` e à procura de uma resposta de turno único (chat completion), podemos criar exemplos usando [este formato sugerido](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) que reflete os requisitos do OpenAI chat completion. Se esperar conteúdo de conversação multi-turno, usaria o [formato de exemplo multi-turno](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) que inclui um parâmetro `weight` para indicar quais as mensagens que devem ser usadas (ou não) no processo de fine-tuning.

Vamos usar o formato mais simples de turno único para o nosso tutorial aqui. Os dados estão no [formato jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) com 1 registo por linha, cada um representado como um objeto formatado em JSON. O excerto abaixo mostra 2 registos como exemplo - veja [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) para o conjunto completo de amostras (10 exemplos) que usaremos para o nosso tutorial de fine-tuning. **Nota:** Cada registo _deve_ ser definido numa única linha (não dividido em várias linhas como é típico num ficheiro JSON formatado)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Num caso de uso real, vai precisar de um conjunto de exemplos muito maior para obter bons resultados - o compromisso será entre a qualidade das respostas e o tempo/custos para o fine-tuning. Estamos a usar um conjunto pequeno para podermos completar o fine-tuning rapidamente e ilustrar o processo. Veja [este exemplo do OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) para um tutorial de fine-tuning mais complexo.


---

### Passo 1.2 Carregar o Seu Conjunto de Dados

Carregue os dados usando a API de Ficheiros [conforme descrito aqui](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Note que, para executar este código, deve ter realizado os seguintes passos primeiro:
 - Instalado o pacote Python `openai` (certifique-se de usar uma versão >=0.28.0 para as funcionalidades mais recentes)
 - Definido a variável de ambiente `OPENAI_API_KEY` com a sua chave de API da OpenAI
Para saber mais, consulte o [Guia de Configuração](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) fornecido para o curso.

Agora, execute o código para criar um ficheiro para carregamento a partir do seu ficheiro JSONL local.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Passo 2.1: Criar o trabalho de Fine-tuning com o SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Passo 2.2: Verificar o Estado do trabalho

Aqui estão algumas coisas que pode fazer com a API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Listar os últimos n trabalhos de fine-tuning
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Obter detalhes de um trabalho de fine-tuning específico
- `client.fine_tuning.jobs.cancel(<job_id>)` - Cancelar um trabalho de fine-tuning
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Listar até n eventos do trabalho
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

O primeiro passo do processo é _validar o ficheiro de treino_ para garantir que os dados estão no formato correto.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Passo 2.3: Acompanhar eventos para monitorizar o progresso


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Passo 2.4: Verificar o estado no Painel OpenAI


Também pode ver o estado visitando o website da OpenAI e explorando a secção _Fine-tuning_ da plataforma. Isto mostrará o estado do trabalho atual, e também lhe permitirá acompanhar o histórico das execuções de trabalhos anteriores. Nesta captura de ecrã, pode ver que a execução anterior falhou, e a segunda execução teve sucesso. Para contexto, isto aconteceu quando a primeira execução usou um ficheiro JSON com registos formatados incorretamente - uma vez corrigido, a segunda execução foi concluída com sucesso e tornou o modelo disponível para uso.

![Fine-tuning job status](../../../../../translated_images/pt-PT/fine-tuned-model-status.563271727bf7bfba.webp)


Pode também ver as mensagens de estado e métricas ao descer mais no painel visual, conforme mostrado:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/pt-PT/fine-tuned-messages-panel.4ed0c2da5ea1313b.webp) |  ![Metrics](../../../../../translated_images/pt-PT/fine-tuned-metrics-panel.700d7e4995a65229.webp)|


---

### Passo 3.1: Recuperar ID & Testar Modelo Ajustado no Código


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Passo 3.2: Carregar e Testar o Modelo Ajustado no Playground

Agora pode testar o modelo ajustado de duas formas. Primeiro, pode visitar o Playground e usar o menu suspenso Models para selecionar o seu modelo recém-ajustado entre as opções listadas. A outra opção é usar a opção "Playground" mostrada no painel de Fine-tuning (veja a captura de ecrã acima) que lança a seguinte vista _comparativa_ que mostra as versões do modelo base e do modelo ajustado lado a lado para uma avaliação rápida.

![Fine-tuning job status](../../../../../translated_images/pt-PT/fine-tuned-playground-compare.56e06f0ad8922016.webp)

Basta preencher o contexto do sistema usado nos seus dados de treino e fornecer a sua pergunta de teste. Vai notar que ambos os lados são atualizados com o contexto e a pergunta idênticos. Execute a comparação e verá a diferença nas respostas entre eles. _Note como o modelo ajustado apresenta a resposta no formato que forneceu nos seus exemplos enquanto o modelo base simplesmente segue o prompt do sistema_.

![Fine-tuning job status](../../../../../translated_images/pt-PT/fine-tuned-playground-launch.5a26495c983c6350.webp)

Vai notar que a comparação também fornece a contagem de tokens para cada modelo, e o tempo levado para a inferência. **Este exemplo específico é simplista e serve para mostrar o processo, mas não reflete um conjunto de dados ou cenário do mundo real**. Pode notar que ambas as amostras mostram o mesmo número de tokens (o contexto do sistema e o prompt do utilizador são idênticos) com o modelo ajustado a levar mais tempo para a inferência (modelo personalizado).

Em cenários do mundo real, não estará a usar um exemplo simples como este, mas sim a ajustar com dados reais (por exemplo, catálogo de produtos para serviço ao cliente) onde a qualidade da resposta será muito mais evidente. Nesse _contexto_, obter uma qualidade de resposta equivalente com o modelo base exigirá mais engenharia de prompt personalizada, o que aumentará o uso de tokens e potencialmente o tempo de processamento relacionado para a inferência. _Para experimentar isto, consulte os exemplos de fine-tuning no OpenAI Cookbook para começar._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Aviso Legal**:
Este documento foi traduzido utilizando o serviço de tradução automática [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos pela precisão, por favor tenha em conta que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autorizada. Para informações críticas, recomenda-se a tradução profissional humana. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações erradas decorrentes do uso desta tradução.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
